Script to download sequence data from NCBI for alignment, cropping and analysis of variation between species.

In [ ]:
# install package requests if you do not yet have it installed
# only need to run this once

import sys
!{sys.executable} -m pip install requests

In [2]:
# read the list of species we are looking for from infile

from Bio import Entrez

infile = 'infiles/fish_species_names.txt'              # or fish_species_names.txt or some other file
outfile_name = 'outfiles/raw_downloads/outfile.fas'    # CHANGE THIS or it will overwrite your last outfile ******************
locus = 'COI'                                          # or cytb or COI, etc.
Entrez.email = ''                        # fill in your e-mail here (should be registered with NCBI)
API_KEY = ''       # use api_key if you have one (makes it faster to search at NCBI)

fish_sealprey = []
with open(infile) as f:                                 
    for line in f:
        fish_sealprey.append(line.strip())
f.close()

In [3]:
from Bio import SeqIO

# Read all fish species into the list
fish_species = [line.strip() for line in open(infile)]

In [6]:
# search NCBI for the locus for all species

# dirty but fast hack for Mac
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

acc_list = []
for fish_species in fish_sealprey:
    # example search term: ("Pterostichus agonus"[Organism] AND 16S[All Fields])
    species_term = '"' + fish_species + '"[Organism] AND ' + locus +'[All Fields]'
    search_handle = Entrez.esearch(db="nucleotide", term=species_term, idtype="acc", usehistory="y", api_key=API_KEY)
    record = Entrez.read(search_handle)
    search_handle.close()
    acc_list += (record["IdList"])

# print the first six accession numbers retrieved
print(acc_list[:6])

['15477', '2659', '146', '7679', '10841', '69436']


In [7]:
# show how many sequence records were retrieved

print(len(acc_list))

11


In [8]:
# obtain DNA sequence data for each accession number from NCBI

from Bio import Entrez
import requests

outfile = open(outfile_name, 'w')
for acc in acc_list:
    handle = Entrez.efetch(db="nucleotide", id=acc, rettype="fasta", retmode="text", api_key=API_KEY)
    # print output to file only if the sequence is not longer than mitochondrial size
    accession = handle.read()
    if len(accession) < 30000:
        outfile.write(accession.replace('N','').replace(' ',''))

print(outfile)
outfile.close()

<_io.TextIOWrapper name='outfiles/raw_downloads/outfile_all_fish_COI.fas' mode='w' encoding='cp1252'>


In [9]:
# produce command line for Clustalw2
# run the command line on a windows command prompt

from Bio.Align.Applications import ClustalwCommandline
cline = ClustalwCommandline("clustalw2", infile=outfile_name)
print(cline)

clustalw2 -infile=outfiles/raw_downloads/outfile_all_fish_COI.fas


now manually crop the alignment (*.aln) to the segment of interest<br>
also remove unalignable sequences<br>

In [ ]:
#haplotype analysis starts here

import numpy as np

def next_haplo_name(hh):
    if hh < 10:
        name = 'H0' + str(hh)
    else: name = 'H' + str(hh)
    return name

fname = 'outfiles/cropped_alignments/harders_16Slong_cropped.aln'
handle = open(fname)
haplos = {} #haplos have names A, B, C etc. as keys and sequence as value
haplo_names = [] #list of haplotype names e.g. H01
sample_names = [] #list of sample names e.g. DH

#fill the fasta outfile with haplotypes
#and fill the lists haplo_names and sample_names
out1 = open('outfiles/haplotype_analysis/sequence_data_haplos.fas','w')
h = 1 #number for the first haplotype
while True:
    line1 = handle.readline() #the name, e.g. DH01-0001
    if not line1: break
    line1 = line1.strip('\n')
    sample = line1[11:20]
    if sample not in sample_names: #check if sample seen before, if not, add it
        sample_names.append(sample)
    line2 = handle.readline() #the DNA sequence
    line2 = line2.upper()#make all uppercase
    if line2 not in haplos.values(): #fill the haplos dictionary
        haplo_name = next_haplo_name(h)
        haplo_names.append(haplo_name)
        h += 1
        haplos[haplo_name] = line2
        line_h = '>' + haplo_name +'\n' + line2
        out1.write(line_h)
handle.close()
out1.close()

In [ ]:
#fill the first csv outfile with the individuals' haplotypes
#will use infile fname = 'sequence_data_cropped.fas' for this
out2 = open('outfiles/haplotype_analysis/sequence_data_ind_haplos.csv','w')
handle = open(fname)
while True:
    line1 = handle.readline() #the name, e.g. DH01-0001
    if not line1: break
    line1 = line1.strip('\n')
    line2 = handle.readline() #the DNA sequence
    for k,v in haplos.items(): #look for the haplotype code
        if v == line2:
            line1 = line1.strip('>').split('-')
            line1 = ','.join(line1)
            line_i = line1 +','+ k + '\n'
            out2.write(line_i)
handle.close()
out2.close()


In [ ]:
#fill csv file (out3) with frequencies per sample
#will use lists haplo_names and sample_names for this
#will also use infile fname = 'sequence_data_ind_haplos.csv' for this
#produce Arlequin infile (out4)
handle2 = open('outfiles/haplotype_analysis/sequence_data_ind_haplos.csv')
out3 = open('outfiles/haplotype_analysis/sequence_data_freqs.csv','w')
out4 = open('outfiles/haplotype_analysis/sequence_data_samples.arp','w')
print('haplotype names: ',haplo_names)
print('sample names: ',sample_names)

In [ ]:
#write the head to the Arlequin file
line_arl = '[Profile]\nTitle="A sample file designed to compute amova"\nNbSamples=' +  str(len(sample_names)) + '\n'\
    'GenotypicData=0\nLocusSeparator=NONE\nDataType=DNA\nCompDistMatrix=0\n[Data]\n'\
    '[[Samples]]\n'
out4.write(line_arl)
freq_table = np.zeros(shape=(len(sample_names),len(haplo_names))) #array for frequency data
freq_table.astype(int)
#fill the first row of csv file out3
#haplo_names
line = ','
for haplo_name in haplo_names:
    line = line + haplo_name + ','
line = line + '\n'
out3.write(line)


In [ ]:
#fill the freq_table array
while True:
    line1 = handle2.readline()
    if not line1: break
    line1 = line1.strip('\n')
    ind_sample = line1[10:19]
    ind_haplo = line1[-3:]
    ind_sample_index = sample_names.index(ind_sample)
    ind_haplo_index = haplo_names.index(ind_haplo)
    freq_table[ind_sample_index,ind_haplo_index] += 1
freq_table = freq_table.astype(int)

sample_sizes = np.sum(freq_table,axis=1).tolist() #list of sample sizes
#write freq_table array to csv file out3
#write the sample data to the Arlequin file
for i in range(len(sample_names)):
    line_i = sample_names[i] + ','
    line_arl = 'SampleName="' + sample_names[i] + '"\nSampleSize=' +\
        str(sample_sizes[i]) + '\nSampleData={\n'
    for j in range(len(haplo_names)):
        line_i = line_i + str(freq_table[i,j]) +','
        line_arl = line_arl + str(haplo_names[j]) + ' ' + str(freq_table[i,j]) + ' ' +\
            haplos[haplo_names[j]]
    line_i = line_i + '\n'
    line_arl = line_arl + '}\n'
    out3.write(line_i)
    out4.write(line_arl)
#write the footer (structure) to the Arlequin file
line_arl = '[[Structure]]\nStructureName="none"\nNbGroups=1\nIndividualLevel=0\n\
    Group={\n'
for k in sample_names:
    line_arl = line_arl + '"' + k + '"\n'
line_arl = line_arl + '}\n'
out4.write(line_arl)

handle2.close()
out3.close()
out4.close()